In [1]:
from tutorial_utils import magma_to_verilog_string, smt_to_smtlib_string

import hwtypes as hw
import magma as m

/home/donovick/work/peak-tutorial/env-peak-tutorial/lib/python3.11/site-packages/ast_tools/immutable_ast.py:10: UserWarning: /home/donovick/work/peak-tutorial/env-peak-tutorial/lib/python3.11/site-packages/ast_tools/immutable_ast.py generated for (3, 9)does not match system version (3, 11)
  warnings.warn(f"{__file__} generated for (3, 9)"


In [2]:
# Program
def add(x, y):
    return x + y

# Python interpretation
PyDataT = hw.BitVector[8]

x = PyDataT(1)
y = PyDataT(2)
results = add(x, y)
print(repr(results))
print('---')

# SMT interpretation
SmtDataT = hw.SMTBitVector[8]

x = SmtDataT(name='x')
y = SmtDataT(name='y')
results = add(x, y)
print(smt_to_smtlib_string(results))
print('---')

# del because jupyter seems to keep references alive longer than it should which breaks SMT variables
del x
del y

# Magma interpretation
MagmaDataT = m.Bits[8]

class Adder(m.Circuit):
    io = m.IO(
        x=m.In(MagmaDataT), 
        y=m.In(MagmaDataT),
        results=m.Out(MagmaDataT),
    )
    io.results @= add(io.x, io.y)


print(magma_to_verilog_string(Adder))
print('---')

BitVector[8](3)
---
(bvadd x y)
---
module coreir_add #(
    parameter width = 1
) (
    input [width-1:0] in0,
    input [width-1:0] in1,
    output [width-1:0] out
);
  assign out = in0 + in1;
endmodule

module Adder (
    input [7:0] x,
    input [7:0] y,
    output [7:0] results
);
wire [7:0] magma_Bits_8_add_inst0_out;
coreir_add #(
    .width(8)
) magma_Bits_8_add_inst0 (
    .in0(x),
    .in1(y),
    .out(magma_Bits_8_add_inst0_out)
);
assign results = magma_Bits_8_add_inst0_out;
endmodule


---


In [3]:
# Program
def abs(x):
    return (x < 0).ite(-x, x)

# Python interpretation
PyDataT = hw.SIntVector[8]
x = PyDataT(-1)

results = abs(x)
print(repr(results))
print('---')

# SMT interpretation
SmtDataT = hw.SMTSIntVector[8]

x = SmtDataT(name='x')
results = abs(x)
print(smt_to_smtlib_string(results))
print('---')
del x

# Magma interpretation
MagmaDataT = m.SInt[8]

class Abs(m.Circuit):
    io = m.IO(x=m.In(MagmaDataT), results=m.Out(MagmaDataT))
    io.results @= abs(io.x)


print(magma_to_verilog_string(Abs))
print('---')

SIntVector[8](1)
---
(ite (bvslt x #b00000000) (bvneg x) x)
---
module coreir_slt #(
    parameter width = 1
) (
    input [width-1:0] in0,
    input [width-1:0] in1,
    output out
);
  assign out = $signed(in0) < $signed(in1);
endmodule

module coreir_neg #(
    parameter width = 1
) (
    input [width-1:0] in,
    output [width-1:0] out
);
  assign out = -in;
endmodule

module coreir_mux #(
    parameter width = 1
) (
    input [width-1:0] in0,
    input [width-1:0] in1,
    input sel,
    output [width-1:0] out
);
  assign out = sel ? in1 : in0;
endmodule

module coreir_const #(
    parameter width = 1,
    parameter value = 1
) (
    output [width-1:0] out
);
  assign out = value;
endmodule

module commonlib_muxn__N2__width8 (
    input [7:0] in_data [1:0],
    input [0:0] in_sel,
    output [7:0] out
);
wire [7:0] _join_out;
coreir_mux #(
    .width(8)
) _join (
    .in0(in_data[0]),
    .in1(in_data[1]),
    .sel(in_sel[0]),
    .out(_join_out)
);
assign out = _join_out;
endmodu

In [4]:
SmtDataT = hw.SMTBitVector[8]

def add_chain(*args):
    n = len(args)
    if n == 0:
        return 0
    else:
        return args[0] + add_n(*args[1:])

def add_tree(*args):
    n = len(args)
    if n == 0:
        return 0
    elif n == 1:
        return args[0]
    else:
        return add_n(*args[:n // 2]) + add_n(*args[n // 2:])

add_n = add_tree

x = SmtDataT(name='x')
y = SmtDataT(name='y')
z = SmtDataT(name='z')
w = SmtDataT(name='w')
results1 = add_n(x)
results2 = add_n(x, y)
results3 = add_n(x, y, z)
results4 = add_n(x, y, z, w)
print(smt_to_smtlib_string(results1))
print(smt_to_smtlib_string(results2))
print(smt_to_smtlib_string(results3))
print(smt_to_smtlib_string(results4))

del x
del y
del z
del w
del results1
del results2
del results3
del results4

x
(bvadd x y)
(bvadd x (bvadd y z))
(bvadd (bvadd x y) (bvadd z w))


In [5]:
SmtDataT = hw.SMTBitVector[8]

def gen_tree_reducer(f, ident=None):
    '''
    f :: T -> T -> T
    ident :: Optional[T]
    '''
    def reducer(*args):
        '''
         *args :: List[T]
        '''
        n = len(args)
        if n == 0:
            if ident is None:
                raise ValueError('cannot reduce empty list')
            return ident
        elif n == 1:
            return args[0]
        else:
            return f(reducer(*args[:n // 2]), reducer(*args[n // 2:]))

    return reducer

def mul(x, y): return x * y

mul_n = gen_tree_reducer(mul, 1)

x = SmtDataT(name='x')
y = SmtDataT(name='y')
z = SmtDataT(name='z')
results1 = mul_n(x)
results2 = mul_n(x, y)
results3 = mul_n(x, y, z)
print(smt_to_smtlib_string(results1))
print(smt_to_smtlib_string(results2))
print(smt_to_smtlib_string(results3))


del x
del y
del z
del results1
del results2
del results3

x
(bvmul x y)
(bvmul x (bvmul y z))
